In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset , DataLoader
from torchvision import models, transforms
from sklearn.model_selection import *
import pandas as pd
import numpy as np
from IPython.display import display
import pickle
from tqdm import tqdm
from collections import Counter



In [2]:
from torch.optim import SGD,Adam, lr_scheduler
from PIL import Image

In [3]:
import os
from os import path
import sys


In [4]:
model = models.inception_v3( pretrained = True)

KeyboardInterrupt: 

In [ ]:
print(model)

In [28]:
model.fc = nn.Linear(in_features = , out_features = 1, bias = True)


In [33]:
class Sign_dataset( Dataset):
    
    def __init__(self, datapath, img_background_color, transform =None):
        
        self.path = datapath
        self.transform = transform
        self.data = pd.read_csv(datapath)
        self.data = self.data[['class', 'address']]
        self.background_color = img_background_color
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        y, imgpath = self.data.loc[idx]
        
        self.img = Image.open(imgpath)
        
        if( self.img.mode == 'RGBA'):
            self.img = self.img.convert('RGB')

        self.expand2square()
        
        if self.img.size[0] < 299:
            tensor = transforms.ToTensor()
            
            pad = nn.ZeroPad2d((299- self.img.size[0])//2)
            self.img = pad(tensor(self.img))
            pil = transforms.ToPILImage()
            self.img = pil(self.img)
            
            
        if( self.transform != None):
            self.img = self.transform(self.img)
            
        if(tuple(self.img.shape )!= (3, 299,299)):
            img = np.zeros((3, 299,299))
            img[0,:, :] = self.img
            img[0,:, :] = self.img
            img[0,:, :] = self.img
            self.img = torch.Tensor(img)
            
        return self.img, y
    
   
    def expand2square(self):
        
        width, height = self.img.size
        
        if width >= height:

            result = Image.new(self.img.mode, ( width, width), self.background_color)
            result.paste(self.img, (0, (width - height) // 2))
            self.img = result

        else:

            result = Image.new(self.img.mode, (height, height), self.background_color)
            result.paste( self.img, ((height - width) // 2,0))
            self.img = result
        

In [46]:
root = "/home/sanchit/Desktop/app/"
folder = 'trainTestData'
train_file = "handcraftedData_train.csv"
test_file = "handcraftedData_test.csv"

batchsize = 256

trainpath = os.path.join(root,folder, train_file)
testpath = os.path.join(root,folder,  test_file)

transformation = transforms.Compose( [transforms.RandomHorizontalFlip(0.5) ,transforms.Resize((2299,299)), \
                                      transforms.Grayscale(3) ,\
                                      transforms.ToTensor(), \
                                      transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

trainLoader = DataLoader(Sign_dataset(trainpath, 100, transformation ), batch_size = batchsize, \
                         shuffle = True)
testLoader = DataLoader(Sign_dataset(testpath, 100, transformation ), batch_size = batchsize, \
                         shuffle = True)
    

In [37]:
### global train_corr, val_corr, train_loss, val_loss
train_corr =[]
val_corr = []
train_loss = []
val_loss = []

def evaluate( model, testloader, criterion, minValLoss ):
     
    with torch.no_grad():
        model.eval()
        runningloss = 0
        runningcorrcoef_ = 0

        for x, y in testloader:

            pred = model(x)
            del x
            
            pred = pred.squeeze()
            loss = criterion(pred, y.float())
            runningloss += loss
            corrcoef_ = np.corrcoef(pred.detach().numpy(), y)[0][1]
            runningcorrcoef_ += corrcoef_
            
            print("eval >>>  corr_coef: {:.3f}   loss: {:.3f}".format( corrcoef_, loss))
            
            
            
        print(pred)
        print(y)   
        avgLoss = runningloss/len(testloader)
        
        if(  avgLoss < minValLoss[0] ):
        # only way to change value of passed varaible in function is to pass mutable object

            minValLoss[0] = avgLoss
            print("Model has Learnt >>>>> saving Model minValLoss: {}".format(minValLoss[0]))
            states = {  'model':model.state_dict(),  
                    }
            
            torch.save( states, '/home/sanchit/Desktop/app/Dump/resnet.pt')
            print(">>> proceeding to next Epoch >>>")
            
        val_corr.append( runningcorrcoef_/len(testloader))
        val_loss.append( runningloss/len(testloader))
    
      
def train( model, trainloader, testloader, lr_, epochs, optimizer, criterion, step_size = 50):
    
    for child in model.children():
        if( child != model.fc ):
            for para in child.parameters():
                para.requires_grad = False
    minValLoss = [1.1798266172409058  ]   
                
    optimizer = optimizer( model.parameters(), lr = lr_ , momentum = 0.9)
    scheduler = lr_scheduler.StepLR(optimizer, step_size= step_size, gamma=0.1)
    from tqdm import tqdm
    for epoch in tqdm(range(epochs)):
        
        runningloss = 0
        runningcorrcoef_ = 0
        
        for x, y in trainloader:
            
            optimizer.zero_grad()
            
            pred = model(x)
            del x
            
            pred = pred.squeeze()
            
            loss = criterion(pred, y.float())
            loss.backward()
            optimizer.step()
            
            pred = pred.detach().numpy()
            corrcoef_ = np.corrcoef( pred, y.float())[0][1]
            runningcorrcoef_ += corrcoef_
            
            runningloss += loss.item()
            
            print("Epoch {}/{} predmean : {:.3f} predstd: {:.3f} predmin: {:.3f} predmax: {:.3f}  corr_coef: {:.3f}   loss: {:.3f}".format( epoch+1, epochs, 
                    pred.mean(), pred.std(), pred.min(), pred.max(), corrcoef_, loss))
           
            
        train_corr.append( runningcorrcoef_/ len(trainloader) )
        train_loss.append( runningloss/ len(trainloader) )
        evaluate( model, testloader, criterion, minValLoss )
        scheduler.step()

            

In [45]:
train(model, trainLoader, testLoader, 0.0001, 30, SGD, nn.MSELoss(), step_size = 20 )

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1/30 predmean : 4.145 predstd: 0.750 predmin: 2.190 predmax: 6.281  corr_coef: 0.509   loss: 1.648
Epoch 1/30 predmean : 4.142 predstd: 0.766 predmin: 1.400 predmax: 6.635  corr_coef: 0.442   loss: 1.705
Epoch 1/30 predmean : 4.241 predstd: 0.753 predmin: 1.855 predmax: 6.852  corr_coef: 0.376   loss: 1.693
Epoch 1/30 predmean : 4.181 predstd: 0.721 predmin: 2.159 predmax: 6.246  corr_coef: 0.487   loss: 1.457
Epoch 1/30 predmean : 4.219 predstd: 0.707 predmin: 2.089 predmax: 6.120  corr_coef: 0.464   loss: 1.646
Epoch 1/30 predmean : 4.263 predstd: 0.806 predmin: 1.706 predmax: 6.319  corr_coef: 0.450   loss: 1.740
Epoch 1/30 predmean : 4.201 predstd: 0.787 predmin: 1.499 predmax: 6.236  corr_coef: 0.425   loss: 1.576
Epoch 1/30 predmean : 4.377 predstd: 0.832 predmin: 1.333 predmax: 6.622  corr_coef: 0.350   loss: 1.878
Epoch 1/30 predmean : 4.247 predstd: 0.797 predmin: 1.676 predmax: 6.580  corr_coef: 0.375   loss: 1.815
Epoch 1/30 predmean : 4.326 predstd: 0.736 predmin: 1.9

  3%|▎         | 1/30 [10:22<5:00:44, 622.24s/it]

eval >>>  corr_coef: 0.429   loss: 1.507
tensor([5.1382, 3.3253, 3.4941, 5.8115, 4.1336, 3.1006, 4.3027, 4.3450, 4.3538,
        5.1718, 3.2526, 5.6282, 4.6584, 2.9401, 4.0858, 3.0524, 4.9119, 4.1917,
        4.2241, 3.4885, 4.9940, 4.5312, 3.0751, 4.5005, 3.5639, 3.5415, 4.0056,
        3.2417, 4.4135, 4.6143, 5.3969, 2.6349, 4.1277, 3.4814, 4.1224, 4.9729,
        5.2800, 3.3876, 4.6870, 3.6010, 4.6258, 5.3014, 4.1761, 3.5087, 4.3828,
        4.2789, 4.1339, 3.1679, 4.2567, 4.2452, 5.1486, 4.5119, 4.4035, 4.4747,
        4.9592, 4.2936, 4.1421, 3.7614, 4.6902, 4.8824, 3.8026, 3.9840, 3.8337,
        2.9331, 3.7575, 5.2365, 4.5168, 5.0583, 3.6437, 4.4173, 6.1570, 3.8330,
        4.7167, 4.9408, 4.5671, 2.8121, 4.6559, 3.6059, 3.1760, 4.6367, 3.3126,
        5.4405, 4.8769, 3.9429, 5.6240, 3.9562, 4.4082, 4.9223, 3.2931, 4.5427,
        3.9160])
tensor([4.7130, 4.9190, 3.5090, 5.6770, 4.1250, 3.7020, 4.0280, 4.8720, 3.6740,
        3.7640, 3.9290, 5.3140, 4.2640, 1.0000, 5.2130, 4.3490

  7%|▋         | 2/30 [21:03<4:53:05, 628.05s/it]

eval >>>  corr_coef: 0.357   loss: 1.755
tensor([3.9838, 4.6023, 4.7909, 5.3036, 5.1510, 3.2124, 3.2827, 3.8498, 5.0112,
        3.8574, 4.1334, 3.7459, 3.3882, 4.5703, 5.6507, 3.1692, 3.3762, 3.1459,
        4.3330, 4.8794, 4.5984, 4.9997, 3.8872, 4.1674, 4.9420, 4.4626, 5.3220,
        5.4913, 3.8774, 4.0862, 3.8084, 4.8040, 4.1598, 4.2682, 3.6981, 4.7520,
        4.0556, 4.0761, 5.0449, 4.4300, 3.7920, 4.7861, 4.5735, 3.9355, 5.2766,
        3.6260, 4.4501, 5.0715, 3.8127, 4.3494, 5.0285, 5.4261, 4.7480, 3.6071,
        4.9938, 3.9676, 4.8744, 3.6764, 4.1304, 3.3561, 5.8954, 4.1471, 4.6025,
        2.8569, 4.2284, 4.3427, 4.2094, 5.1437, 3.7190, 4.6116, 3.6789, 4.5590,
        4.5985, 3.8866, 4.4869, 2.3540, 5.9065, 4.6482, 3.3642, 4.2354, 3.7638,
        5.6907, 4.1951, 3.3056, 4.7711, 4.8742, 4.2664, 4.7026, 3.9252, 5.6420,
        4.2067])
tensor([4.4570, 3.4170, 3.7020, 5.5300, 5.7150, 3.5600, 4.1250, 1.0000, 6.0000,
        2.8780, 4.1110, 5.5950, 3.5860, 1.0000, 4.0390, 6.0000

 10%|█         | 3/30 [31:15<4:40:24, 623.13s/it]

eval >>>  corr_coef: 0.313   loss: 1.444
tensor([1.8299, 3.4897, 4.0127, 3.7263, 5.4141, 4.2952, 5.2757, 5.0130, 4.7176,
        4.9273, 3.4480, 4.6262, 5.6162, 4.0773, 4.2351, 4.4607, 3.9650, 5.5221,
        3.0491, 4.5767, 3.0162, 4.3602, 5.0643, 3.6448, 4.4630, 4.3418, 4.1283,
        3.8036, 4.9279, 4.3289, 5.8585, 6.3820, 5.0236, 4.4968, 4.2211, 4.5919,
        4.0379, 4.3079, 3.0796, 4.1605, 4.7174, 5.1000, 5.7576, 4.6741, 4.6600,
        3.2128, 3.4397, 4.9162, 4.9489, 4.5073, 3.9322, 4.5090, 4.5364, 5.1401,
        3.7328, 3.5476, 3.5222, 5.2399, 3.9611, 3.8361, 3.3647, 6.4995, 3.1777,
        2.6901, 4.2415, 3.7748, 4.2238, 2.9853, 5.2076, 4.4728, 4.4095, 4.2951,
        3.8126, 5.1063, 4.4300, 4.3447, 3.6686, 4.2633, 4.5925, 4.4266, 3.1854,
        4.0244, 4.8613, 5.6626, 4.5314, 3.8111, 4.3372, 4.5044, 4.6555, 4.4960,
        5.8967])
tensor([1.0000, 5.0040, 4.6630, 3.7790, 5.0210, 3.9280, 3.7480, 5.2600, 7.0000,
        5.0000, 3.2030, 4.2530, 4.6460, 3.5380, 4.6680, 3.7290

 10%|█         | 3/30 [45:20<6:48:03, 906.80s/it]


KeyboardInterrupt: 

In [40]:
from torch import cuda

In [11]:
!python3 nima/cli.py validate_model --path_to_model_weight ./pretrain-model.pth --path_to_save_csv $PATH_TO_CSV  --path_to_images $PATH_TO_IMAGES --batch_size $BATCH_SIZE --num_workers $NUM_EPOCH

python3: can't open file 'nima/cli.py': [Errno 2] No such file or directory
